In [1]:
# List of attributes of stations
#     -name
#     -prov
#     -lat
#     -lon
#     -elevation
#     -station_id
#     -start_date
#     -end_date
#     -climate_id
#     -wmo_id
#     _tc_id

This is a quick demo of my progress on this sprint<br>
In this demo each weather station attempts to grab the last<br>
daily mean tempurature that was released by that station from the internet.<br><br>
However there is not much error handling so some stations data<br>
may look odd, or be empty

In [2]:
# Dependencies
!pip install requests --user
!pip install beautifulsoup4 --user
!pip install tqdm --user
!pip install ipyleaflet --user
!pip install mapboxgl --user
!jupyter nbextension enable --py --user ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [23]:
html_table = '''<html>
<head>
<style>
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100;
}

td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
}

tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
</head>
<body>

<h2>HTML Table</h2>

<table>
  <tr>
    <th>Station</th>
    <th>Latitude</th>
    <th>Longitude</th>
  </tr>
  <tr>
    <td>%s</td>
    <td>%s</td>
    <td>%s</td>
  </tr>
</table>

</body>
</html>'''

In [24]:
html_table[100:120]

' width: 100;\n}\n\ntd, '

In [14]:
from ipywidgets import HTML

table = HTML()
table.hold_sync()
table

HTML(value='')

In [6]:
table.value = html_table

In [25]:
from scipy.spatial import cKDTree
def update_table(kdtree, table, pos, Stations, Markers, Popups):
    global html_table
    query = kdtree.query(pos)
    index = query[1]

    print(Popups[index].value)
    if 'yes' in Popups[index].value:
        name = get_station_attr(Stations[index], 'name')
        lat = get_station_attr(Stations[index], 'lat')
        lon = get_station_attr(Stations[index], 'lon')
        print(name)
        print(lat)
        print(lon)
        table.value = (html_table % (name, lat, lon))

In [16]:
import time
from tqdm import tqdm
from ipywidgets import HTML, Label
from ipyleaflet import Marker, Map, Popup, MarkerCluster
from pickle_to_txt import read_stations, get_station_attr
from map_utils import linear_station_find, init_kdtree, kdtree_station_find

current_time = lambda: int(round(time.time() * 1000))

# Initailize the map of Alberta
center_view = (52.585538631877306, 245.45105824782084)
alberta = Map(center=center_view, zoom=6, close_popup_on_click=False)

# Fill three databases of information for later
Stations = read_stations('alberta_stations.txt')
Markers = [None] * len(Stations)
Popups = [None] * len(Stations)

kdTree = init_kdtree(Stations)

# Define the callback function that will be used
last_time = current_time()
blacklist = ['mouseover', 'click', 'dblclick', 'mousedown']

def update_info(type, event, coordinates):
    if type in blacklist:
        pass
    elif type == 'mousemove':
        # Need a cleaner way to implement a static var
        global last_time, kdTree
        
        # Only updates once in every 0.05 sec
        if current_time()-last_time > 50:

            #linear_station_find(Stations, Markers, Popups, coordinates)
            kdtree_station_find(kdTree, Stations, Markers, Popups, coordinates)
            last_time = current_time()
            
    elif type == 'mouseup':
        print("Mouse up")
        update_table(kdTree, table, coordinates, Stations, Markers, Popups)

# Put a marker on the map for each station
for i, station in enumerate(tqdm(Stations)):
    marker = Marker(location=(float(get_station_attr(station, 'lat')), 
                              float(get_station_attr(station, 'lon'))), 
                    draggable=False)
    Markers[i] = marker
#     Popups[i] = HTML(value='<p>--         </p>\n<p>-----         \n             </p>',
#                      placeholder='',
#                      description='')

#    Markers[i].popup = Popups[i]
    
marker_cluster = MarkerCluster(markers=Markers)
alberta.add_layer(marker_cluster)
alberta.on_interaction(update_info)


100%|██████████| 1482/1482 [00:15<00:00, 93.77it/s]


In [26]:
alberta

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[53.66547308629858, 244.02172878384593], close_popup_on_click=False, east=254.79931667447093, layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), MarkerCluster(markers=(Marker(draggable=False, location=[51.7, 244.46666666666667], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[54.28333333333333, 248.96666666666667], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.18333333333333, 250.23333333333332], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.066136111111106, 250.31696944444445], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.06666666666667, 250.13333333333333], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.083333333333336, 250.28333333333333], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.19611388888889, 250.16889166666667], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.03333333333333, 250.3], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.5, 247.41666666666666], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[59.166666666666664, 242.33333333333334], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[53.71666666666667, 242.56666666666666], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[49.05, 249.31666666666666], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[51.31666666666667, 247.88333333333333], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[49.026666666666664, 246.0511111111111], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[53.11333444444445, 249.18111277777777], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[52.8, 246.81666666666666], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[56.11666666666667, 249.78333333333333], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[52.36666666666667, 247.3], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[52.38333333333333, 247.16666666666666], options=['alt', 'clickable', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'title', 'z_index_offset']), Marker(draggable=False, location=[52

Mouse up
<p>MISTANUSK CREEK</p>
<p>Latitude: 54.592<br>Longitude: 240.00<br>End Date: </p><html>
<body>
<p id="show_in_table" hidden>no</p>
<button type="button" onclick="document.getElementById('show_in_table').innerHTML = 'yes'">Show in table</button>
 
</body>
</html>
MISTANUSK CREEK
54.59277777777778
240.00916666666666
Mouse up
<p>MISTANUSK CREEK</p>
<p>Latitude: 54.592<br>Longitude: 240.00<br>End Date: </p><html>
<body>
<p id="show_in_table" hidden>no</p>
<button type="button" onclick="document.getElementById('show_in_table').innerHTML = 'yes'">Show in table</button>
 
</body>
</html>
MISTANUSK CREEK
54.59277777777778
240.00916666666666
Mouse up
<p>MISTANUSK CREEK</p>
<p>Latitude: 54.592<br>Longitude: 240.00<br>End Date: </p><html>
<body>
<p id="show_in_table" hidden>no</p>
<button type="button" onclick="document.getElementById('show_in_table').innerHTML = 'yes'">Show in table</button>
 
</body>
</html>
MISTANUSK CREEK
54.59277777777778
240.00916666666666
Mouse up
<p>FONTAS LO</p>

Hello World
